# Accessing the 3DCityDB

This notebook gives basic examples of how to interact with the 3DCityDB through the **DBLayer** package.

## Connecting to the database

Accessing the database is achieved through class `DBAccess`, which allows to connect to an instance of the extended 3DCityDB by calling function `connect_to_citydb` using an instance of `PostgreSQLConnectionInfo`:

In [1]:
from dblayer import *

# Define connection parameters.
connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'testdb'
    )

# Create access point.
db_access = DBAccess()

# Connect to database.
db_access.connect_to_citydb( connect )

For the purpose of this notebook you might want to use an empty 3DCityDB. To this end, the next two lines erase all data from an existing database. **Only execute the next two lines if you really want to erase all data from your database!**

In [2]:
db_access.cleanup_citydb_schema()
db_access.cleanup_simpkg_schema()

## Storing data to the database

After a successful connection, class `DBAccess` offers several functions enabling a user-friendly interaction with the database.
For instance, an important feature is that the SQL functions of the extended 3DCityDB can be called to insert new objects.
The following code snippet demonstrates how a new heat pump object is inserted using SQL function `insert_heat_pump`.

In [3]:
from dblayer.func.func_citydb_view_nrg import *

# Add a new entry to the databse using SQL function "insert_heat_pump".
hp_id = db_access.add_citydb_object(
    insert_heat_pump,
    name = 'HEATPUMP_01',
    nom_effcy = 1.2,
    effcy_indicator = 'COP'
    )

Finally, these changes are committed permanently to the database.

In [4]:
db_access.commit_citydb_session()

**Note**: Package `DBLayer` also provides specialized functions for storing domain-specific data. Notebooks [PowerGridModelWriteDB](./PowerGridModelWriteDB.ipynb), [ThermalNetworkModelWriteDB](./ThermalNetworkModelWriteDB.ipynb) and [GasNetworkModelWriteDB](./GasNetworkModelWriteDB.ipynb) show how they work.

## Reading data from the database

Class `DBAccess` provides several ways of accessing and querying data from the database.
The following example shows how heat pump data can be retrieved from an *updatable view*.
First, class `HeatPump` is associated with the view `citydb_view.nrg8_conv_system_heat_pump`.
This association is then used to refine conditions for querying the database.
Finally, function `get_citydb_objects` is called to retrieve data from the specified view with the specified query conditions.

In [5]:
# Map table "citydb_view.nrg8_conv_system_heat_pump" to Python class HeatPump.
HeatPump = db_access.map_citydb_object_class(
  'HeatPump',
  schema = 'citydb_view',
  table_name = 'nrg8_conv_system_heat_pump'
  )

# Use the attributes of class HeatPump to define query conditions.
conditions = [
  HeatPump.name == 'HEATPUMP_01',
  HeatPump.nom_effcy == 1.2
  ]

# Query the database using the conditions defined above.
heatpumps = db_access.get_citydb_objects(
  'HeatPump',
  conditions = conditions
  )

# Retrieve data from query result.
print( 'heatpump_id = {}'.format( heatpumps[0].id ) )
print( 'effcy_indicator = {}'.format( heatpumps[0].effcy_indicator ) )

heatpump_id = 1
effcy_indicator = COP


For more detailed examples of how to access data please refer to notebook [PowerGridModelWriteDB](./PowerGridModelWriteDB.ipynb).